# Bien choisir les ressources
Il y a plusieurs ressources informatiques qui vous sont disponibles à
l'Alliance de recherche numérique du Canada :
* Calcul haute performance
  * Béluga, Cedar, Graham, Narval, Niagara
* Stockage
  * Temporaire, projet, *nearline*, dépôt de données de recherche
* Infonuagique
  * Arbutus, Béluga, Cedar, Graham

Le principal but de ce chapitre est de vous permettre d'analyser
vos besoins en **ressources de calcul haute performance**, et ce,
dans le but de choisir les ressources nécessaires pour votre projet.

## Rappel - le calcul haute performance
* Lorsqu'il est question de lancer une **grande quantité de calculs**
(séquentiels ou parallèles) ou de **traitements de données**,
l'utilisation d'une grappe de calcul haute performance devient nécessaire.
* Puisque les **ressources sont partagées** et en grande demande,
chaque tâche doit être soumise à un ordonnanceur de tâches.
* Il devient donc nécessaire **d'estimer à l'avance les ressources**
qui seront réservées lors de l'exécution d'un calcul.

Dans cette section, on cherche à évaluer le comportement d'une tâche de calcul
afin d'estimer et d'optimiser les ressources à réserver sur la grappe de calcul.

### Objectif - construction d'un script de tâche Slurm
Un script de tâche pour [l'ordonnanceur Slurm](https://slurm.schedmd.com/documentation.html) est typiquement un script Bash dans lequel on retrouve :
* Le [shebang](https://fr.wikipedia.org/wiki/Shebang) en toute première ligne. Par exemple : `#!/bin/bash`
* Les options `#SBATCH` en entête pour les besoins de la tâche. Les options en entête seront lues par la commande de soumission de tâche [`sbatch`](https://slurm.schedmd.com/sbatch.html)
* [Chargement des modules](https://docs.computecanada.ca/wiki/Utiliser_des_modules) requis
* Les commandes Bash qui seront exécutées automatiquement sur des processeurs réservés pour la tâche

Par exemple : [`scripts/mpi-allo.sh`](https://github.com/calculquebec/cip201-serveurs-calcul/blob/main/scripts/mpi-allo.sh)

```Bash
cat scripts/mpi-allo.sh
```
```
#!/bin/bash
#SBATCH --ntasks=10
#SBATCH --mem-per-cpu=1000M
#SBATCH --time=0-00:10

mpirun printenv HOSTNAME OMPI_COMM_WORLD_RANK OMPI_COMM_WORLD_SIZE
```

Notre documentation à cet effet débute à la page : [Exécuter des tâches](https://docs.computecanada.ca/wiki/Running_jobs/fr)

## Analyse des calculs à petite échelle
Lorsqu'une tâche de calcul est en cours d'exécution, vous pouvez surveiller différentes métriques :
* Utilisation CPU (et GPU, s'il y a lieu)
* Mémoire résidente (réellement utilisée)
* Mémoire virtuelle (allouée)
* Les accès au stockage

### Sous Windows
* [Gestionnaire des tâches Windows](https://fr.wikipedia.org/wiki/Gestionnaire_des_t%C3%A2ches_Windows)
* Pour le faire afficher :
  * Raccourcis clavier Ctrl+Alt+Suppr ou cliquer sur la barre des tâches avec le bouton droit
  * Cliquer sur l'option *Gestionnaire des tâches*

![Aperçu du gestionnaire des tâches Windows](images/win-task-manager.png)

### Sous Mac OS
* [Moniteur d’activité](https://support.apple.com/fr-ca/guide/activity-monitor/actmntr1001/mac)
* Pour le faire afficher :
  * Démarrer l'application à partir des *Applications et Utilitaires* de Mac OS
  * Sinon, utiliser le raccourcis clavier Commande+Espace et taper les premières lettres de "Moniteur d'activité" pour pouvoir sélectionner cette application

![Aperçu du moniteur d'activité Mac OS](https://help.apple.com/assets/5FDCF1894EB74318147EC0CF/5FDCF18A4EB74318147EC0D6/fr_CA/ad6337d66061aa27122e75521960fc5a.png)

### Sous Linux
Dans un terminal Unix, on peut utiliser :
* La [commande `top`](https://man7.org/linux/man-pages/man1/top.1.html) (`q` pour quitter)

![Capture de top](images/linux-top.png)

* La [commande `htop`](https://man7.org/linux/man-pages/man1/htop.1.html) (`q` pour quitter)

![Capture de htop](images/linux-htop.png)

### Sur la grappe de calcul directement
Avec votre accès par défaut, vous avez un compte de calcul `def-*` de base
vous permettant de lancer des tâches de test.

#### Ressources utilisées d'une tâche terminée
```Bash
ssh login1
...
```
**Note** : pour accéder aux grappes de calcul en production,
il vaut mieux utiliser [une paire de clés SSH](https://docs.computecanada.ca/wiki/Using_SSH_keys_in_Linux/fr).

Pour soumettre un script, on fait :
```Bash
sbatch scripts/blastn-gen-seq.sh
```

Pour voir l'état de la tâche, on ferait :
```Bash
squeue -u $USER  # ou 'sq'
```

Avec la commande [`sacct`](https://slurm.schedmd.com/sacct.html), on peut obtenir un tableau de nos tâches exécutées depuis minuit.
```Bash
sacct
```

Avec la commande [`seff`](https://docs.computecanada.ca/wiki/Running_jobs/fr#T.C3.A2ches_termin.C3.A9es),
on peut obtenir un court rapport d'exécution de tâche.
Ce rapport inclut une mesure du temps écoulé, une mesure du temps CPU
et une mesure de consommation maximale de la mémoire-vive.
Des valeurs d'efficacité en pourcentages sont données pour les cycles CPU
et la mémoire-vives en fonction des quantités réservées.
```Bash
seff <No_tâche>
```

#### Ressources utilisées par une tâche CPU en cours
Étant donné un certain calcul matriciel dans le script Python 
[`scripts/crunch.py`](https://github.com/calculquebec/cip201-serveurs-calcul/blob/main/scripts/crunch.py) :

```Bash
less scripts/crunch.py   # q pour quitter
```

Lors d'une tâche interactive, on peut utiliser `top` et `htop` pour surveiller les ressources utilisées :

```Bash
# Tâche interactive et installation de modules Python
salloc --gres=gpu:1 --ntasks-per-node=4 --mem=8000M --time=0:9:0
less scripts/installer/cupy_tmp.sh  # q pour quitter
bash scripts/installer/cupy_tmp.sh
```

```Bash
# Exécution avec un processeur
less scripts/crunch-1cpu.sh  # q pour quitter
bash scripts/crunch-1cpu.sh  # q pour quitter

# Exécution avec 4 processeurs
less scripts/crunch-4cpu.sh  # q pour quitter
bash scripts/crunch-4cpu.sh  # q pour quitter

# Comparer les résultats
grep sec *.log

exit  # Pour revenir à login1
```

#### Ressources utilisées par une tâche GPU en cours
```Bash
# Tâche interactive et installation de modules Python
salloc --gres=gpu:1 --ntasks-per-node=4 --mem=8000M --time=0:9:0
bash scripts/installer/cupy_tmp.sh
```

* Pour Windows et Mac OS, il existe des outils propriétaires permettant de visualiser en temps réel l'utilisation du GPU. Veuillez vous référer au site Web du manufacturier de votre GPU pour les détails
* Sous Linux, il y a d'abord la [commande `nvidia-smi`](https://developer.nvidia.com/nvidia-system-management-interface)

```Bash
nvidia-smi
```

![Capture nvidia-smi](images/nvidia-smi.png)

* Il existe aussi un projet [`nvtop`](https://github.com/Syllo/nvtop) permettant de visualiser l'utilisation d'un ou plusieurs GPUs dans un terminal

```Bash
# Exécution avec 1 GPU
less scripts/crunch-1gpu.sh  # q pour quitter
bash scripts/crunch-1gpu.sh  # q pour quitter

# Regarder le résultat
less tg.log    # q pour quitter

exit  # Pour revenir à login1
```

![Capture nvtop](https://raw.githubusercontent.com/Syllo/nvtop/master/screenshot/NVTOP_ex1.png)

#### Comparer la vitesse CPU vs GPU
Avant d'utiliser massivement les GPUs d'une grappe de calcul, il faut tout d'abord que l'application ou l'algorithme puisse démontrer une "bonne performance" en utilisant plusieurs processeurs en parallèle.

Quelques définitions :
* **Temps écoulé** = temps d'exécution total que l'on perçoit et non le temps CPU
* **Accélération** = (temps avec un processeur) / (temps avec parallélisme)
* **Efficacité** = (Accélération) / (nombre de processeurs)

Le coût d'un noeud GPU étant de quatre à cinq fois supérieur à celui d'un noeud régulier, l'utilisation d'un seul GPU doit permettre une accélération d'au moins quatre fois (4x) la vitesse de huit (8) à douze (12) processeurs.
* **Accélération** = (temps avec 8 à 12 processeurs) / (temps avec un accélérateur)

#### Exercice - Calcul d'accélération et d'efficacité
En supposant les résultats suivants pour un programme parallèle lancé sur $n$ processeurs :

```Bash
grep sec t*.log
```

|$n$ proc.|temp (s)|
|:-------:|:------:|
|    1    |  9.876 |
|    4    |  5.220 |

Étant donné le script [`scripts/calc-acc-eff.sh`](https://github.com/calculquebec/cip201-serveurs-calcul/blob/main/scripts/calc-acc-eff.sh) :
```Bash
bash scripts/calc-acc-eff.sh 1:9.876 4:5.220
bash scripts/calc-acc-eff.sh 1:9.876 8:0.035
```

## Extrapoler les ressources nécessaires
### Efficacité cible du calcul parallèle
* Une efficacité de 80%, voire 90%, devrait être un seuil minimal pour les tâches parallèles. Or, il existe un **nombre maximal de processeurs** à utiliser pour respecter ce seuil :
  * Voir la figure dans la page [loi d'Amdahl](https://fr.wikipedia.org/wiki/Loi_d%27Amdahl)
  * Matériel en extra [ici](extra/loi-Amdahl.ipynb)
* On vise aussi une consommation en mémoire-vive de l'ordre de 80% de ce qui est demandé à l'ordonnanceur Slurm

Rappel - vous pouvez obtenir ces pourcentages via les commandes `sacct -X` (surtout pour obtenir les numéros de tâches) et `seff`. Les valeurs à considérer sont :
* `CPU Utilized` et `CPU Efficiency`
* `Memory Utilized` et `Memory Efficiency`

### Taille des données et nombre de fichiers à traiter
Pour un calcul donné, il y a deux métriques de stockage à considérer :
1. La **quantité** totale en octets (ou Go)
1. Le **nombre** total de fichiers

Pour obtenir ces informations :
* **Sous Windows** : dans l'explorateur Windows (raccourcis clavier : Windows + E)
  * Sélectionner un dossier ou plusieurs fichiers
  * Bouton droit de la souris -> *Propriétés*

![Windows data properties](images/win-data-size.png)

* **Sous Mac OS** : dans *Finder*
  * Sélectionner un dossier ou plusieurs fichiers
  * Bouton droit de la souris -> *Get Info*
  * Autrement : avec l'affichage *Par liste*
    * [Activer *Calculer toutes les tailles*](https://www.solutionenligne.org/comment-afficher-taille-dossiers-fichiers-dans-finder-mac-os/)

* **Sous Linux** :
  * L'environnement graphique peut offrir le même genre d'outils, mais tout dépend de la distribution et du bureau.
  * La commande `du -bs DOSSIER` (`b` : taille apparente en octets, `s` : somme totale) calcule récursivement et affiche la taille totale en octets. La taille apparente est celle qui importe lors d'un transfert ou d'une sauvegarde de données.
  * La commande `find DOSSIER | wc -l` compte récursivement et affiche le nombre de fichiers et de sous-dossiers.

#### Stockage en mémoire selon les types de base
En ayant une idée de la taille des données à traiter,
il devient possible d'estimer l'espace que les données prendront en mémoire-vive.

* Dans un fichier texte, **chaque caractère** prend de un (1) à deux (2) octets, en moyenne.
Cependant, pour certaines langues, l'encodage
[UTF-8](https://fr.wikipedia.org/wiki/UTF-8#Description)
peut se rendre jusqu'à quatre (4) octets par caractère.
Pour les langues latines et germaniques, on peut considérer **deux (2) octets** par caractère.
Par exemple, dans une session Python :

```Bash
python
```

```Python
>>> import sys
>>> euro = ""
>>> sys.getsizeof(euro)
49
>>> euro = "Euro"
>>> sys.getsizeof(euro)
53
>>> euro = "€"
>>> sys.getsizeof(euro)
76
>>> euro *= 2  # Donc "€€"
>>> sys.getsizeof(euro)
78
>>> quit()  # ou Ctrl+D pour sortir
```

* Les **nombres entiers** prennent typiquement 2 octets (16 bits), 4 octets (32 bits) ou 8 octets (64 bits) chacun. Tout dépend de la [plage de valeurs souhaitée](https://en.wikipedia.org/wiki/C_data_types#Main_types) :
  * 2 octets : ~65 milles valeurs de $0$ à $65535$, ou de $-32767$ à $32767$
  * 4 octets : ~4 milliards de valeurs de 0 à $(2^{32}-1)$ ou de $-(2^{31}-1)$ à $(2^{31}-1)$
  * 8 octets : ~18 trillions de valeurs de 0 à $(2^{64}-1)$ ou de $-(2^{63}-1)$ à $(2^{63}-1)$

```Bash
module load python scipy-stack
python
```

```Python
>>> import sys
>>> import numpy as np
>>> cube = np.zeros((100,100,100), dtype=np.int64)
>>> sys.getsizeof(cube)
8000136
>>> cube = np.zeros((100,100,100), dtype=np.int32)
>>> sys.getsizeof(cube)
4000136
>>> np.iinfo(np.int16)
iinfo(min=-32768, max=32767, dtype=int16)
```

* Les **nombres à virgule flottante** prennent typiquement 4 octets (32 bits) ou 8 octets (64 bits) chacun, mais on voit de plus en plus [différents types de données à 2 octets (16 bits)](https://en.wikipedia.org/wiki/Bfloat16_floating-point_format) dans des applications d'apprentissage-machine. Il se peut néanmoins que les données soient initialement en simple ou double précision:
  * [simple précision](https://en.wikipedia.org/wiki/Single-precision_floating-point_format) : 4 octets, une résolution de 23 bits (~7 décimales), une échelle de 8 bits (${10}^{-38}$ à ${10}^{38}$)
  * [double précision](https://en.wikipedia.org/wiki/Double-precision_floating-point_format) : 8 octets, une résolution de 52 bits (~16 décimales), une échelle de 11 bits (${10}^{-308}$ à ${10}^{308}$)

```Python
>>> cube = np.ndarray((100,100,100), dtype=np.float32)
>>> cube[0,0,0] = np.pi
>>> print(cube[0,0,0])
3.1415927
>>> print(cube[0,0,0], np.pi)
3.1415927 3.141592653589793
```

* Certains langages utilisent systématiquement **8 octets** (64 bits) par nombre.
* [Certains compilateurs et certaines bibliothèques](https://en.wikipedia.org/wiki/Quadruple-precision_floating-point_format) peuvent calculer des valeurs représentées avec 128 bits [ou plus](https://gmplib.org/).
* Pour les nombres complexes, on multiplie l'espace mémoire par deux (2).

Exemple de calcul de l'espace-mémoire :
```Python
>>> nb_matrices = 3        # Trois matrices C = prod_mat(A, B)
>>> taille = 25000         # Matrices carrées
>>> octets_par_nombre = 8  # Double précision
>>> memoire = nb_matrices * taille*taille * octets_par_nombre
>>> memoire / 1000**3
>>>
>>> quit()  # ou Ctrl+D pour sortir
```

#### La complexité des algorithmes
La question qui se pose : en augmentant la ou les dimensions du problème, quelles devraient être la durée du calcul et la consommation en mémoire-vive?
Une [analyse de la complexité de l'algorithme principal](https://fr.wikipedia.org/wiki/Analyse_de_la_complexit%C3%A9_des_algorithmes)
permettrait de connaître l'ordre $O$ du calcul en fonction de la taille $n$ des données :

* $O(n)$: proportionnel à $n$
* $O(n*m)$: représente un calcul à deux (2) dimensions indépendantes
* $O(n^3)$: calcul d'ordre cubique
* $O(n*m*k^2)$: par exemple, un filtre de taille $k*k$ sur une image $n*m$
* $O(n*log(n))$: typique de certains
[algorithmes de tri](https://fr.wikipedia.org/wiki/Algorithme_de_tri#Comparaison_des_algorithmes)
où il y a $n$ éléments à trier en $log_2(n)$ étapes.

Une analyse détaillée du code (s'il est disponible) n'est pas nécessaire pour déterminer le type de calcul qui est fait.
* Vous pouvez vous inspirer des données en entrées pour deviner l'ordre du calcul principal. 
* Vous pouvez mesurer le temps d'exécution en fonction de la taille du problème. En extrapolant les résultats, il serait possible de prévoir le comportement du programme sur une grappe de calcul.

#### Exercice - Complexité de l'algorithme
Dans cet exercice, il est question d'inverser une matrice de valeurs aléatoires de taille $n*n$ :
* Soumettre le script [`scripts/inv-mat.sh`](https://github.com/calculquebec/cip201-serveurs-calcul/blob/main/scripts/inv-mat.sh) avec la commande `sbatch` :

```Bash
sbatch scripts/inv-mat.sh
```

* Explorer les scripts Python
[`scripts/inv-mat.py`](https://github.com/calculquebec/cip201-serveurs-calcul/blob/main/scripts/inv-mat.py) et 
[`scripts/inv-mat.sh`](https://github.com/calculquebec/cip201-serveurs-calcul/blob/main/scripts/inv-mat.sh) :

```Bash
cat scripts/inv-mat.py
cat scripts/inv-mat.sh
```

* Suivre l'évolution du calcul avec `squeue -u $USER`, environ aux 30 secondes
* Le résultat sera sauvegardé dans le fichier `temps_inv.csv`
* Analyse avec Python, Pandas et Numpy dans le script [`scripts/inv-mat-pred.py`](https://github.com/calculquebec/cip201-serveurs-calcul/blob/main/scripts/inv-mat-pred.py) :

```Bash
module load gcc python scipy-stack

cat scripts/inv-mat-pred.py
python scripts/inv-mat-pred.py temps_inv.csv
```

## Principales différences entre les grappes de calcul
* À propos des grappes :

| | [Béluga](https://docs.computecanada.ca/wiki/B%C3%A9luga) | [Cedar](https://docs.computecanada.ca/wiki/Cedar/fr) | [Graham](https://docs.computecanada.ca/wiki/Graham/fr) | [Narval](https://docs.computecanada.ca/wiki/Narval) | [Niagara](https://docs.computecanada.ca/wiki/Niagara/fr) |
|-----------------------:|:---------:|:---------:|:---------:|:------------:|:----------:|
| **Mise en production** | Mars 2019 | Juin 2017 | Juin 2017 | Octobre 2021 | Avril 2018 |
|              **Ville** | Montréal  |  Burnaby  | Waterloo  |   Montréal   |   Toronto  |
|           **Province** |  Québec   |    C.-B.  |  Ontario  |    Québec    |   Ontario  |

* Nombre de processeurs (coeurs CPU) selon le cas :

| Processeur Intel/AMD  | [Béluga](https://docs.computecanada.ca/wiki/B%C3%A9luga) | [Cedar](https://docs.computecanada.ca/wiki/Cedar/fr) | [Graham](https://docs.computecanada.ca/wiki/Graham/fr) | [Narval](https://docs.computecanada.ca/wiki/Narval) | [Niagara](https://docs.computecanada.ca/wiki/Niagara/fr) |
|----------------------:|:--------:|:--------:|:--------:|:---------:|:---------:|
|      Broadwell (avx2) |          | 724 * 32 | 983 * 32 |           |           |
|      Skylake (avx512) | 802 * 40 | 640 * 48 |          |           | 1548 * 40 |
| Cascade Lake (avx512) |          | 768 * 48 |  72 * 44 |           |  468 * 40 |
|      EPYC Rome (avx2) |          |          |          | 1142 * 64 |           |

| Mémoire par proc. | [Béluga](https://docs.computecanada.ca/wiki/B%C3%A9luga) | [Cedar](https://docs.computecanada.ca/wiki/Cedar/fr) | [Graham](https://docs.computecanada.ca/wiki/Graham/fr) | [Narval](https://docs.computecanada.ca/wiki/Narval) | [Niagara](https://docs.computecanada.ca/wiki/Niagara/fr) |
|-------:|:-----:|:-----:|:-----:|:-----:|:-----:|
|  2400M |  6400 |       |       |       |       |
|  4000M |       | 86016 | 28896 | 70976 |       |
|  4400M |       |       |  3168 |       |       |
|  4800M | 23560 |       |       |       | 80960 |
|  8000M |       |  3072 |  1792 |       |       |
| 16000M |       |   768 |   768 |       |       |
| 19200M |  2120 |       |       |       |       |
| 32000M |       |       |       |  2112 |       |
| 48000M |       |   768 |   192 |       |       |
| 96000M |       |   128 |       |       |       |

* [Nombre de GPUs](https://docs.computecanada.ca/wiki/Using_GPUs_with_Slurm/fr) selon le cas :

| Accélérateurs | [Béluga](https://docs.computecanada.ca/wiki/B%C3%A9luga) | [Cedar](https://docs.computecanada.ca/wiki/Cedar/fr) | [Graham](https://docs.computecanada.ca/wiki/Graham/fr) | [Mist (Power9)](https://docs.scinet.utoronto.ca/index.php/Mist) | [Narval](https://docs.computecanada.ca/wiki/Narval) |
|----------------:|:---:|:---:|:---:|:---:|:---:|
| NVIDIA P100 12G |     | 456 | 320 |     |     |
| NVIDIA P100 16G |     | 128 |     |     |     |
|   NVIDIA T4 16G |     |     | 144 |     |     |
| NVIDIA V100 16G | 688 |     |  54 |     |     |
| NVIDIA V100 32G |     | 768 |  16 | 216 |     |
| NVIDIA A100 40G |     |     |     |     | 632 |

* Réseau haute-performance et ordonnancement :

| | [Béluga](https://docs.computecanada.ca/wiki/B%C3%A9luga) | [Cedar](https://docs.computecanada.ca/wiki/Cedar/fr) | [Graham](https://docs.computecanada.ca/wiki/Graham/fr) | [Narval](https://docs.computecanada.ca/wiki/Narval) | [Niagara](https://docs.computecanada.ca/wiki/Niagara/fr) |
|------------------------:|:----------:|:-----------:|:----------:|:-----------:|:----------:|
|        Connexion rapide | InfiniBand |   OmniPath  | InfiniBand | InfiniBand  | InfiniBand |
|               Topologie |  En arbre  |   En arbre  |  En arbre  |  En arbre   | DragonFly+ |
|     Taille îlots (proc) | 640 à 1200 | 1024 à 1536 |    1024    | 3072 à 3584 |    17280   |
|     Facteur de blockage |   max 5:1  |   max 2:1   |   max 8:1  |  max 4.7:1  |   max 2:1  |
| Granularité des tâches  | /proc /GPU |  /proc /GPU | /proc /GPU |  /proc /GPU |   /noeud   |
|         Durée maximale  |   7 jours  |   28 jours  |  28 jours  |   7 jours   |   1 jour   |

* Stockage : le tout sera décrit au dernier chapitre.

## Points à retenir
* Prévoir les **paramètres d'une tâche Slurm**
  * Nombre de processeurs (CPU) et de noeuds de calcul
  * Nombre d'accélérateurs (GPU)
  * Quantité de mémoire-vive (RAM)
  * Temps du calcul (`JJ-H:M` ou `H:M:S`)
* Différents **outils pour surveiller** les ressources utilisées
  * `time` et autres bibliothèques de mesure du temps écoulé
  * `top`, `htop`, `nvtop`, `nvidia-smi`
  * `sacct`, `seff`
  * `du -bs`, `find | wc -l` et autres outils du système d'exploitation
* On vise une **efficacité de 80%** pour les tâches parallèles CPU
  * L'accélération avec un accélérateur (GPU) doit être significative (>4x)
* Le choix de la grappe dépend des besoins de chaque type de calcul